In [ ]:
!git clone https://github.com/huggingface/transformers

import os
import pandas as pd 
os.chdir("./transformers")
!pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 80215, done.
remote: Total 80215 (delta 0), reused 0 (delta 0), pack-reused 80215
Receiving objects: 100% (80215/80215), 62.69 MiB | 24.87 MiB/s, done.
Resolving deltas: 100% (57435/57435), done.
Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 15.4 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 636 kB 67.5 MB/s 
     |███████████

In [ ]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/Datasets/kaggle_rock_beginnings.csv" "kaggle_rock_beginnings.csv"

Mounted at /content/drive


In [ ]:
#alter LM Head model to path with finetuned weights
model = TFGPT2LMHeadModel.from_pretrained("/content/drive/My Drive/NLP/GPT-2/Model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/My Drive/GPT-2/Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv("kaggle_rock_beginnings.csv")

In [ ]:
beginnings = df["beginnings"].values.tolist() 
beginnings = beginnings[:20] # check the results for top 20 songs
beginnings

['if i never ever ever get to see you again\nremember it was you that made me smile\nyou satisfy the need you created\nnot for the first time could be for real\nand',
 'under some dirty words on a dirty wall\neating takeout by myself\ni played the shows\ngot back in the van and put the walkman on\nand you were playing\nin some other dive a',
 "the ceiling's low the walls are thin\nthe little stars upon your skin\ni cannot sleep and i'm wound too tight\nthe morning comes and i feel alright\nand take me down home on",
 'i strap my sneakers on and sneak into the garage\ntoday im relaxin gotta get my scooter on\nim gonna roll away im gonna hit the day\nim gonna scoot away my troubles',
 "during the day\nshe's a full time typist\nat the computer\nand on the phone\nduring the day\nhe's a automechanic\npunchin' a time clock\nand livin' alone\nbut at night\nin the dark\nwhen the sun goes down\nyes at",
 "i can't see you mama\nbut i know you're always there\nooh to touch and to feel you mama\no

In [ ]:
encoded_beginnings = [tokenizer.encode("<|endoftext|>"+str(s),return_tensors='tf') for s in beginnings]

In [ ]:
"""
top_p=0.92
temperature= 0.85
top_k = 125
num_beams = 1
early_stopping = True
no_repeat_ngram_size = 0
length_penalty = 1.2
max_length = 200
min_length = 180
repetition_penalty = 1.1
"""

'\ntop_p=0.92\ntemperature= 0.85\ntop_k = 125\nnum_beams = 1\nearly_stopping = True\nno_repeat_ngram_size = 0\nlength_penalty = 1.2\nmax_length = 200\nmin_length = 180\nrepetition_penalty = 1.1\n'

In [ ]:
#experiment with more beams that yielded better coherence of the text but far shorter songs
"""
top_p = 0.95
top_k = 125
num_beams = 5
temperature = 0.85
early_stopping = True
repetition_penalty = 1.3 
no_repeat_ngram_size = 8
length_penalty = 1.2  # 1.1 bis 2.0

"""

'\ntop_p = 0.95\ntop_k = 125\nnum_beams = 5\ntemperature = 0.85\nearly_stopping = True\nrepetition_penalty = 1.3 \nno_repeat_ngram_size = 8\nlength_penalty = 1.2  # 1.1 bis 2.0\n\n'

In [ ]:
generatedEndings = []
top_p=0.92
temperature= 0.85
top_k = 125
num_beams = 1
early_stopping = True
no_repeat_ngram_size = 0
length_penalty = 1.2
max_length = 200
min_length = 180
repetition_penalty = 1.1

for beginning in encoded_beginnings:
    generated_text_samples = model.generate(
      beginning, 
      max_length=max_length, #average rock song length 180-200 words, 1024 max sequence length of gpt2
      min_length=min_length, 
      length_penalty = length_penalty,
      num_return_sequences=1,
      temperature=temperature, # 0.8 - 0.9 seems to be the optimal range 
      early_stopping=early_stopping, #true better 
      repetition_penalty = repetition_penalty, #low repetition penalty to encurage repetitions in the chorus
      eos_token_id = tokenizer.eos_token_id,
      pad_token_id = tokenizer.eos_token_id,
      num_beams = num_beams, # increasing the number of beams increases the song coherence + grammar but drastically decreases the song length 
      do_sample = True, 
      top_k = top_k, 
      top_p = top_p
    )
    generatedEndings.append(generated_text_samples[0])



In [ ]:
generatedEndings = [tokenizer.decode(ending, skip_special_tokens=True) for ending in generatedEndings]

In [ ]:
generatedEndings_stripped = [" ".join(ending.split(" ")[30:]) for ending in generatedEndings]

In [ ]:
print("beginning: \n" + beginnings[0] + "\n" + "generated ending:\n" + generatedEndings_stripped[0])

beginning: 
if i never ever ever get to see you again
remember it was you that made me smile
you satisfy the need you created
not for the first time could be for real
and
generated ending:
i know there is a place in your heart
for you to take me back again
you're the one who made my world come down
i wanna be around
now that's what i'm here for you
i'd do anything to make you smile
it's all you and me oh baby
even though i just can't get enough of you
my heart still beats in time with you
no more holding on to you
in this world where no one cares
wherever you may be
we'll always be together
i'll never let you go
i'll keep holding on to you
if i never ever ever ever get to see you again
remember it was you that made me smile
you satisfy the need you created
not for the first time could be for real


In [ ]:
generatedEndings = pd.DataFrame(generatedEndings_stripped)
generatedEndings = generatedEndings.rename(columns={0: "endings"})
generatedEndings.to_csv("/content/drive/My Drive/Datasets/kaggle_rock_endings_generated.csv")